In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
cc = CasosConfirmados()
# cc.read()
# cc.save()
cc.load()
notifica = Notifica()
# notifica.download_todas_notificacoes()
# notifica.save()
notifica.load()
print(cc.shape())
print(notifica.shape())

(581195, 10614, 425720, 141444)
(2296606, 1055528, 10936, 1227993, 2149)


In [3]:
casosc = cc.get_casos()
casosn = notifica.get_casos()
print(casosn.shape)

(2296606, 55)


In [4]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
print(len(idx_casos_confirmados))
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
print(len(idx_casos_confirmados))
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()
print(len(idx_casos_confirmados))

idx_casos_confirmados = set(idx_casos_confirmados)
print(len(idx_casos_confirmados))

casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.shape

531996
1033553
1506798
553285


(27910, 23)

In [5]:
cchr = casosc.loc[casosc['hash_resid'].isin(casosn['hash_resid'])].copy()
ccha = casosc.loc[casosc['hash_atend'].isin(casosn['hash_atend'])].copy()
cchd = casosc.loc[casosc['hash_diag'].isin(casosn['hash_diag'])].copy()

In [6]:
ccha.loc[ccha['hash_atend']=='WELYNGTONBEVILAQUADASILVA28PARANAVAI']

,ordem,identificacao,ibge_resid,ibge_atend,paciente,sexo,idade,mun_resid,mun_atend,rs,laboratorio,data_diagnostico,data_comunicacao,data_1o_sintomas,obito,data_obito,data_com_obito,recuperado,ativo,excluir,hash_resid,hash_atend,hash_diag
581137,581138,662792,4118402,NaN,WELYNGTON BEVILAQUA DA SILVA,M,28,PARANAVAI,PARANAVAI,14,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),2021-02-11,2021-02-11,NaT,0,NaT,NaT,NaN,NaN,NaN,WELYNGTONBEVILAQUADASILVA28PARANAVAI,WELYNGTONBEVILAQUADASILVA28PARANAVAI,WELYNGTONBEVILAQUADASILVA11022021


In [7]:
cnhr = casosn.loc[casosn['hash_resid'].isin(casosc['hash_resid'])].copy()
cnha = casosn.loc[casosn['hash_atend'].isin(casosc['hash_atend'])].copy()
cnhd = casosn.loc[casosn['hash_diag'].isin(casosc['hash_diag'])].copy()

In [8]:
cnha.loc[cnha['hash_atend']=='WELYNGTONBEVILAQUADASILVA28PARANAVAI']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado,manter
1505943,2356267,WELYNGTON BEVILAQUA DA SILVA,1992-04-03,NEUSELI BEVILAQUA DA SILVA,086.749.109-47,3,CPF INFORMADO,28,M,99,IGNORADO,0,,41,412670,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,NaT,NaT,0,2,TESTE RAPIDO,167,TESTE RAPIDO ANTIGENO (IMUNOCROMATOGRAFIA),12,REAGENTE,NaT,NaT,NaT,1,ABERTA,2,1,NOTIFICA COVID-19,41,411840,2021-02-11,2021-02-11,2021-02-11,TAMBOARA,PR,14,PARANAVAI,PR,WELYNGTONBEVILAQUADASILVA28TAMBOARA,WELYNGTONBEVILAQUADASILVA28PARANAVAI,WELYNGTONBEVILAQUADASILVANEUSELIBEVILAQUADASILVA,WELYNGTONBEVILAQUADASILVA03041992,WELYNGTONBEVILAQUADASILVA11022021,0,0


In [9]:
mchr = pd.merge(cchr,cnhr[['id','hash_resid','manter']],on='hash_resid',how='left')
mcha = pd.merge(ccha,cnha[['id','hash_atend','manter']],on='hash_atend',how='left')
mchd = pd.merge(cchd,cnhd[['id','hash_diag','manter']],on='hash_diag',how='left')

print(mchr.shape)
print(mcha.shape)
print(mchd.shape)
print(mchr.shape[0] + mcha.shape[0] + mchd.shape[0])

(625174, 25)
(578885, 25)
(481549, 25)
1685608


In [10]:
mccn = mchr.copy()
mccn = mccn.append(mcha.loc[~mcha['ordem'].isin(mchr['ordem'])].copy())
mccn = mccn.append(mchd.loc[~mchd['ordem'].isin(mchr['ordem'].append(mcha['ordem']))].copy())

In [11]:
mccn = mccn.drop(index=mccn.loc[(mccn.duplicated('ordem',keep=False)) & (mccn['manter']==0)].index)
mccn.shape

(552072, 25)

In [13]:
casosc.loc[~casosc['ordem'].isin(mccn['ordem'])].shape

(31910, 23)

In [14]:
mccn.shape[0] + casosc.loc[~casosc['ordem'].isin(mccn['ordem'])].shape[0]

583982

In [16]:
writer = pd.ExcelWriter("mccn.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

mccn.to_excel(writer,sheet_name='mccn',index=None)
casosc.loc[~casosc['ordem'].isin(mccn['ordem'])].to_excel(writer,sheet_name='nmccn',index=None)
casosc.loc[~casosc['ordem'].isin(mccn['ordem'])].groupby('data_comunicacao')[['ordem']].count().to_excel(writer,sheet_name='totais',index=None)

writer.save()
writer.close()